In [ ]:
#%pip install python-bioformats tifffile javabridge tkinter numpy jpype1

cellpose ver2.0.1を入れて作った。oirを読むためにjavaをパソコンに入れる必要があるよ。解析PC09に環境がある

In [ ]:
import bioformats
import javabridge
import tifffile
import numpy as np
import tkinter as tk
from tkinter import filedialog
import os
import matplotlib.pyplot as plt
from cellpose import models, io, plot

# ======= 初期ディレクトリ設定（任意で変更） =======
initial_dir = r"."

# ======= ファイル選択ダイアログ =======
root = tk.Tk()
root.withdraw()  # メインウィンドウ非表示

file_path = filedialog.askopenfilename(
    title="ファイルを選択してください",
    initialdir=initial_dir,
    # filetypes=[("Olympus Image Files", "*.oir")]
)

if not file_path:
    print("ファイルが選択されませんでした。処理を中止します。")
    exit()

#ファイルの種類による条件分岐
if  file_path.endswith(".oir"):

    
# JVM 再起動対応（まず kill_vm を安全に実行）
    try:
        javabridge.kill_vm()
    except:
        pass  # JVMが起動していなければ無視

    # JVM 起動
    try:
        javabridge.start_vm(class_path=bioformats.JARS,
                            max_heap_size="512m",
                            run_headless=True)
    except RuntimeError as e:
        print("⚠ Java VM の起動に失敗しました:", e)
        exit()

    # ======= 画像の読み込み =======
    with bioformats.ImageReader(file_path) as reader:
        metadata = bioformats.get_omexml_metadata(file_path)
        #z:深さ、t:タイムライン c:チャンネル
        size_z = reader.rdr.getSizeZ()
        size_t = reader.rdr.getSizeT()
        size_c = reader.rdr.getSizeC()
        size_x = reader.rdr.getSizeX()
        size_y = reader.rdr.getSizeY()

        print(f"画像サイズ: T={size_t}, Z={size_z}, C={size_c}, X={size_x}, Y={size_y}")

        # 例: チャンネル0, 時間0のみ読み込む
        image_stack = []
        for c in range(size_c):
            plane = reader.read(z=0, t=0, c=c, rescale=False)
            image_stack.append(plane)

        imgs = np.array(image_stack)

        # ======= Java仮想マシン停止 =======
    javabridge.kill_vm()
    print("java終了")
elif file_path.endswith(".tif")or file_path.endswith(".tiff"):
    imgs = io.imread(file_path)


"""
imgsの引数はチャンネル!
"""


In [ ]:

# # ======= 出力ファイル名の設定 =======
# output_path = []
# output_path.append(os.path.splitext(file_path)[0] + ".tiff") #pathのoirをtiffに変換している。

# # ======= TIFFとして保存 =======
# tifffile.imwrite(output_path[0], image_stack)
# print(f"変換完了: {output_path}")


In [ ]:
# Cellpose モデル
model = models.Cellpose(model_type='cyto2')

# 入力画像（チャンネルごと）→ 例: [img_ch1, img_ch2]

all_masks = []

for channel in range(len(imgs)):
    img = imgs[channel]  # チャンネル画像を取り出す

    # グレースケール画像 → Cellposeに渡すときのチャンネル設定
    channels = [0, 0]

    # セグメンテーションの実行
    masks, flows, styles, diams = model.eval(img,
                                              diameter=16,
                                              channels=channels,
                                              flow_threshold=0.4,
                                              cellprob_threshold=0.0)

    # 結果を保存
    all_masks.append(masks)
    print(f'diameter:{diams}')

    # セグメンテーション結果を表示
    fig = plt.figure(figsize=(10, 5))
    plot.show_segmentation(fig, img, masks, flows[0], channels=channels)
    plt.tight_layout()
    plt.show()

    # 細胞数（最大ラベル数）を出力
    num_cells = np.max(masks)
    print(f"チャンネル{channel}で検出された細胞数: {num_cells}")


In [ ]:
def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum() #重なっている部分の面積
    union = np.logical_or(mask1, mask2).sum() #両方の領域の合計面積。
    if union == 0:
        return 0
    return intersection / union

#重なりをみたチャンネルを選んで
masks_ch1 = all_masks[1]
masks_ch2 = all_masks[2]

iou_threshold = 0.2  # 共陽性とみなすIOUのしきい値（0.2～0.5が一般的）

# チャンネル1の細胞ID（0は背景なので除く）
ids_ch1 = np.unique(masks_ch1) 
ids_ch1 = ids_ch1[ids_ch1 > 0]

# チャンネル2の細胞ID
ids_ch2 = np.unique(masks_ch2)
ids_ch2 = ids_ch2[ids_ch2 > 0]

common_positive_count = 0

#チャンネル１の細胞を取り出し、チャンネル２のそれぞれの細胞とどれくらい重なっているかを順番に比較

positive_masks=[]
for id1 in ids_ch1:
    mask1 = masks_ch1 == id1 #id1の細胞のmaskを格納
    for id2 in ids_ch2:
        mask2 = masks_ch2 == id2 #id2の細胞のmaskを格納
        iou = compute_iou(mask1, mask2)
        if iou > iou_threshold:
            common_positive_count += 1
            positive_masks.append(np.logical_and(mask1, mask2))
            break  # 1つマッチすればOK（重複カウントを防ぐため）

print(f"\n共陽性の細胞数（IOU>{iou_threshold}）: {common_positive_count}")


# positive_masks はTrue/Falseの2Dマスクのリスト
# 画像サイズ（例：元マスクと同じサイズ）
if not positive_masks==[]:
    h, w = positive_masks[0].shape
    # 0で初期化したラベルマスクを作成
    combined_mask = np.zeros((h, w), dtype=int)

    for i, mask in enumerate(positive_masks, start=1):
        combined_mask[mask] = i  # Trueの部分にiを割り当て


    fig = plt.figure(figsize=(10, 5))
    img = imgs[1]
    plot.show_segmentation(fig, img, combined_mask, flows[0], channels=channels)
    plt.tight_layout()
    plt.show()
